In [1]:
# necessary packages #

#using Pkg
#Pkg.add("Distances")
using Distributions
using Random
using Distances
using LinearAlgebra
using SparseArrays
using IterativeSolvers
using ProgressMeter
using JLD2
using DataFrames

In [2]:
include("../../../util2.j")

colnorm (generic function with 1 method)

In [3]:
# Packages for diagnostic test #

using StatsPlots
using MCMCChains
using PrettyTables

In [4]:
@time @load "../../data/LMC_data/misalign_res_data.jld"

┌ Warning: type OrderedCollections.OrderedDict{Symbol,Any} does not exist in workspace; reconstructing
└ @ JLD2 /home/lu/.julia/packages/JLD2/KjBIK/src/data.jl:1153


  4.067105 seconds (7.97 M allocations: 2.046 GiB, 30.51% gc time)


39-element Array{Symbol,1}:
 :N            
 :U_indx       
 :U_ind        
 :S_indx       
 :S_ind        
 :M_ind        
 :M            
 :R            
 :NR           
 :NS           
 :NM           
 :NU           
 :coords_total 
 ⋮             
 :parts        
 :parts_train  
 :parts_test   
 :CVNN_train   
 :CVNN_test    
 :CVnnIndx_col 
 :CVnnIndx_row 
 :CVMnnIndxLU  
 :CVMnnIndx    
 :CVMnndists   
 :CVIndex_train
 :CVRSMPEind   

In [5]:
using RData
@time data_cleaned = load("../../data/rawdata/data_cleaned_small_expanded.RData", convert = true)["data_cleaned_small"];
@time hold_index_set = load("../../data/rawdata/hold_index_small_expanded.RData", convert = true);

  9.073358 seconds (33.27 M allocations: 2.083 GiB, 9.80% gc time)
  0.066804 seconds (49.89 k allocations: 7.060 MiB)


In [6]:
DataFrames.describe(data_cleaned)

,variable,mean,min,median,max
,Symbol,Float64,Float64,Float64,Float64
1,x,-1.04265e7,-1.11193e7,-1.03937e7,-1.00078e7
2,y,3.83786e6,3.33608e6,3.81839e6,4.44757e6
3,scaled_x,0.6928,0.0,0.725548,1.11149
4,scaled_y,0.501779,0.0,0.482309,1.11149
5,NDVI,0.240122,-0.00954541,0.192437,0.666598
6,EVI,0.150172,-0.00400802,0.122129,0.605463
7,red_reflectance,0.144424,0.0047,0.1452,0.3892
8,NIR_reflectance,0.243485,0.0116,0.2383,0.6165
9,blue_reflectance,0.0752273,0.0001,0.0755,0.2


In [7]:
Y_ord = convert(Matrix, data_cleaned[:, [:NDVI, :red_reflectance]]);
X_ord = [fill(1.0, (N, 1)) convert(Matrix, data_cleaned[:, 28:28])];

In [8]:
data_cleaned = nothing; hold_index_set = nothing; GC.gc();

In [9]:
Sys.free_memory()/(2^20*1024)

7.097515106201172

In [10]:
# priors #
p = 2; q = 2;
μβ = fill(0.0, p, q); inv_Vr = spzeros(p, p); # set Vr^{-1} be zero for the simulation...
νΣ = q + 1; ΨΣ = [[1.0 0.0]; [0.0 1.0]];
inv_Lr = spzeros(p, p); Lμβ = inv_Lr * μβ;

In [11]:
# using equation (12), we have
inv_Vstar = cholesky(X_ord[R, :]'X_ord[R, :] + inv_Vr)
μstar = inv_Vstar \ (X_ord[R, :]'Y_ord[R, :] + inv_Vr * μβ)
invUμstar = inv_Vstar.U * μstar;
Ψstar = ΨΣ + Y_ord[R, :]'Y_ord[R, :] + transpose(μβ) * inv_Vr * μβ -
    invUμstar'invUμstar;
νstar = νΣ + length(R);
Lstar = cholesky(inv(inv_Vstar));

In [12]:
μstar

2×2 Array{Float64,2}:
  0.251458  0.13951  
 -0.133822  0.0603949

In [13]:
# Preallocation for posterior samples #
N_sam = 500;
β_sam = Array{Float64, 3}(undef, p, q, N_sam);
Σ_sam = Array{Float64, 3}(undef, q, q, N_sam);
Y_U_sam = Array{Float64, 3}(undef, NU, q, N_sam);   # missing value
Y_miss_sam1 = Array{Float64, 2}(undef, length(M_ind[1]), N_sam);
Y_miss_sam2 = Array{Float64, 2}(undef, length(M_ind[2]), N_sam);
cholΣ_sam = Array{Float64, 2}(undef, q, q);

In [14]:
# generate the posterior samples #
Random.seed!(123);
prog = Progress(N_sam, 1, "Computing initial pass...", 50)
for l in 1:N_sam
    Σ_sam[:, :, l] = rand(InverseWishart(νstar, Ψstar), 1)[1];    # sample Σ
    cholΣ_sam = cholesky(Σ_sam[:, :, l]);
    β_sam[:, :, l] = (Lstar.L * rand(Normal(), p, q)) * 
                    cholΣ_sam.U + μstar;          # sample γ
    Dic_diag_cond_std = Dict(2^0 => sqrt(Σ_sam[2, 2, l] - Σ_sam[2, 1, l] / 
            Σ_sam[:, :, l][1, 1] * Σ_sam[1, 2, l]), 
        2^1 => sqrt(Σ_sam[1, 1, l] - Σ_sam[1, 2, l] / Σ_sam[2, 2, l] * Σ_sam[2, 1, l]));
    
    Dic_diag_cond_cor = Dict(2^0 => Σ_sam[2, 1, l] / Σ_sam[1, 1, l], 
        2^1 => Σ_sam[1, 2, l] / Σ_sam[2, 2, l]);
    
    Y_U_sam[:, :, l] = X_ord[U_indx, :] * β_sam[:, :, l] + 
        rand(Normal(), NU, q) * cholΣ_sam.U;
    Y_miss_sam1[:, l] = X_ord[M_ind[1], :] * β_sam[:, 1, l] + 
        (Σ_sam[1, 2, l] / Σ_sam[2, 2, l]) * (Y_ord[M_ind[1], 2] - 
        X_ord[M_ind[1], :]* β_sam[:, 2, l]) + 
    rand(Normal(0, Dic_diag_cond_std[2]), length(M_ind[1]));
    Y_miss_sam2[:, l] = X_ord[M_ind[2], :] * β_sam[:, 2, l] + 
        (Σ_sam[2, 1, l] / Σ_sam[1, 1, l]) * (Y_ord[M_ind[2], 1] - 
        X_ord[M_ind[2], :] * β_sam[:, 1, l]) + 
    rand(Normal(0, Dic_diag_cond_std[1]), length(M_ind[2]));
    next!(prog) # monitor the progress
end

Computing initial pass...100%|██████████████████████████████████████████████████| Time: 0:00:08


In [15]:
Y_m1_pos_qt = Array{Float64, 2}(undef, length(M_ind[1]), 3);
Y_m1_pos_mean = Array{Float64, 1}(undef, length(M_ind[1]));
Y_m1_pos_var = Array{Float64, 1}(undef, length(M_ind[1]));
for i in 1:length(M_ind[1])
    Y_m1_pos_qt[i, :] = quantile(Y_miss_sam1[i, :], [0.025, 0.5, 0.975]);
    Y_m1_pos_mean[i] = mean(Y_miss_sam1[i, :]);
    Y_m1_pos_var[i] = var(Y_miss_sam1[i, :]);
end
Y_m2_pos_qt = Array{Float64, 2}(undef, length(M_ind[2]), 3);
Y_m2_pos_mean = Array{Float64, 1}(undef, length(M_ind[2]));
Y_m2_pos_var = Array{Float64, 1}(undef, length(M_ind[2]));
for i in 1:length(M_ind[2])
    Y_m2_pos_qt[i, :] = quantile(Y_miss_sam2[i, :], [0.025, 0.5, 0.975]);
    Y_m2_pos_mean[i] = mean(Y_miss_sam2[i, :]);
    Y_m2_pos_var[i] = var(Y_miss_sam2[i, :]);
end
Y_U_pos_qt = Array{Float64, 3}(undef, NU, q, 3);
Y_U_pos_mean = Array{Float64, 2}(undef, NU, q);
Y_U_pos_var = Array{Float64, 2}(undef, NU, q);
for j in 1:q
    for i in 1:NU
        Y_U_pos_qt[i, j, :] = quantile(Y_U_sam[i, j, :], [0.025, 0.5, 0.975]);
        Y_U_pos_mean[i, j] = mean(Y_U_sam[i, j, :]);
        Y_U_pos_var[i, j] = var(Y_U_sam[i, j, :]);
    end
end

In [35]:
# calculate root mean square predictive error #
AE = fill(0.0, 2);
AE[1] = sum(abs.(Y_ord[U_indx, 1] - Y_U_pos_mean[:, 1]));
AE[2] = sum(abs.(Y_ord[U_indx, 2] - Y_U_pos_mean[:, 2]));
AE[1] = AE[1] + sum(abs.(Y_m1_pos_mean - Y_ord[M_ind[1], 1]));
AE[2] = AE[2] + sum(abs.(Y_m2_pos_mean - Y_ord[M_ind[2], 2]));

round.([(AE[1] / (NU + length(M_ind[1]))) (AE[2] / 
            (NU + length(M_ind[2]))) (sum(AE) / 
            (2 * NU + length(M_ind[1]) + length(M_ind[2])))], digits = 4)

1×3 Array{Float64,2}:
 0.0582  0.0281  0.0432

In [36]:
# calculate root mean square predictive error #
SPE = fill(0.0, 2);
SPE[1] = sum((Y_ord[U_indx, 1] - Y_U_pos_mean[:, 1]).^2);
SPE[2] = sum((Y_ord[U_indx, 2] - Y_U_pos_mean[:, 2]).^2);
SPE[1] = SPE[1] + sum((Y_m1_pos_mean - Y_ord[M_ind[1], 1]).^2);
SPE[2] = SPE[2] + sum((Y_m2_pos_mean - Y_ord[M_ind[2], 2]).^2);

round.(sqrt.([(SPE[1] / (NU + length(M_ind[1]))) (SPE[2] / 
            (NU + length(M_ind[2]))) (sum(SPE) / 
            (2 * NU + length(M_ind[1]) + length(M_ind[2])))]), digits = 4)

1×3 Array{Float64,2}:
 0.074  0.0359  0.0582

In [39]:
# CRPS
CRPS_U = [(sqrt(Y_U_pos_var[i, j]) * ( 1 /sqrt(π) - 
        2 * pdf(Normal(), (Y_ord[U_indx[i], j] - Y_U_pos_mean[i, j]) / sqrt(Y_U_pos_var[i, j])) -
        ((Y_ord[U_indx[i], j] - Y_U_pos_mean[i, j]) / sqrt(Y_U_pos_var[i, j])) * 
        (2* cdf(Normal(), (Y_ord[U_indx[i], j] - Y_U_pos_mean[i, j]) / sqrt(Y_U_pos_var[i, j])) - 1 )))
        for i in 1:NU, j in 1:q];
CRPS1 = [(sqrt(Y_m1_pos_var[i]) * ( 1 /sqrt(π) - 
        2 * pdf(Normal(), (Y_ord[M_ind[1][i], 1] - Y_m1_pos_mean[i]) / sqrt(Y_m1_pos_var[i])) -
        ((Y_ord[M_ind[1][i], 1] - Y_m1_pos_mean[i]) / sqrt(Y_m1_pos_var[i])) * 
        (2* cdf(Normal(), (Y_ord[M_ind[1][i], 1] - Y_m1_pos_mean[i]) / sqrt(Y_m1_pos_var[i])) - 1 )))
        for i in 1:length(M_ind[1])];
CRPS2 = [(sqrt(Y_m2_pos_var[i]) * ( 1 /sqrt(π) - 
        2 * pdf(Normal(), (Y_ord[M_ind[2][i], 1] - Y_m2_pos_mean[i]) / sqrt(Y_m2_pos_var[i])) -
        ((Y_ord[M_ind[2][i], 2] - Y_m2_pos_mean[i]) / sqrt(Y_m2_pos_var[i])) * 
        (2* cdf(Normal(), (Y_ord[M_ind[2][i], 2] - Y_m2_pos_mean[i]) / sqrt(Y_m2_pos_var[i])) - 1 )))
        for i in 1:length(M_ind[2])];

round.([((sum(CRPS_U[:, 1]) + sum(CRPS1))/(NU + 
            length(M_ind[1]))) ((sum(CRPS_U[:, 2]) + sum(CRPS2))/(NU + 
            length(M_ind[2]))) ((sum(CRPS_U) + sum(CRPS1) + sum(CRPS2))/(2*NU + 
            length(M_ind[1]) + length(M_ind[2])))], digits = 5)

1×3 Array{Float64,2}:
 -0.0414  -0.01052  -0.02596

In [29]:
# count the covarage of 95% CI #
count = fill(0.0, 2);
for j in 1:q
    for i in 1:NU
        count[j] = count[j] + 
        ((Y_U_pos_qt[i, j, 1] < Y_ord[U_indx[i], j]) && 
            (Y_U_pos_qt[i, j, 3] > Y_ord[U_indx[i], j]))
    end
end
for i in 1:length(M_ind[1])
    count[1] = count[1] + 
        ((Y_m1_pos_qt[i, 1] < Y_ord[M_ind[1][i], 1]) && 
            (Y_m1_pos_qt[i, 3] > Y_ord[M_ind[1][i], 1]))
end
for i in 1:length(M_ind[2])
    count[2] = count[2] + 
        ((Y_m2_pos_qt[i, 1] < Y_ord[M_ind[2][i], 2]) && 
            (Y_m2_pos_qt[i, 3] > Y_ord[M_ind[2][i], 2]))
end
round.([(count[1] / (NU + length(M_ind[1]))) (count[2] / 
            (NU + length(M_ind[2]))) (sum(count) / (2 * NU + 
                length(M_ind[1]) + length(M_ind[2])))], digits = 4)

1×3 Array{Float64,2}:
 0.9526  0.9547  0.9537

In [40]:
#INT
INT_U = [((Y_U_pos_qt[i, j, 3] - Y_U_pos_qt[i, j, 1]) + 
    (2 / 0.05)*(Y_U_pos_qt[i, j, 1] - Y_ord[U_indx[i], j]) * (Y_ord[U_indx[i], j] < Y_U_pos_qt[i, j, 1]) + 
    (2 / 0.05)*(Y_ord[U_indx[i], j] - Y_U_pos_qt[i, j, 3]) * (Y_ord[U_indx[i], j] > Y_U_pos_qt[i, j, 3]))
    for i in 1:NU, j in 1:q];
INT1 = [((Y_m1_pos_qt[i, 3] - Y_m1_pos_qt[i, 1]) + 
    (2 / 0.05)*(Y_m1_pos_qt[i, 1] - Y_ord[M_ind[1][i], 1]) * (Y_ord[M_ind[1][i], 1] < Y_m1_pos_qt[i, 1]) + 
    (2 / 0.05)*(Y_ord[M_ind[1][i], 1] - Y_m1_pos_qt[i, 3]) * (Y_ord[M_ind[1][i], 1] > Y_m1_pos_qt[i, 3]))
    for i in 1:length(M_ind[1])];
INT2 = [((Y_m2_pos_qt[i, 3] - Y_m2_pos_qt[i, 1]) + 
    (2 / 0.05)*(Y_m2_pos_qt[i, 1] - Y_ord[M_ind[2][i], 2]) * (Y_ord[M_ind[2][i], 2] < Y_m2_pos_qt[i, 1]) + 
    (2 / 0.05)*(Y_ord[M_ind[2][i], 2] - Y_m2_pos_qt[i, 3]) * (Y_ord[M_ind[2][i], 2] > Y_m2_pos_qt[i, 3]))
    for i in 1:length(M_ind[2])];


round.([((sum(INT_U[:, 1]) + sum(INT1))/(NU + 
            length(M_ind[1]))) ((sum(INT_U[:, 2]) + sum(INT2))/(NU + 
            length(M_ind[2]))) ((sum(INT_U) + sum(INT1) + sum(INT2))/(2*NU + 
            length(M_ind[1]) + length(M_ind[2])))], digits = 5)

1×3 Array{Float64,2}:
 0.34868  0.17283  0.26077

In [27]:
N_Inf_burn = 1;
summary_table = Array{Float64, 2}(undef, 7, 4);
summary_table[1, :] = vcat(mean(β_sam[1, 1, N_Inf_burn:N_sam]),
    quantile(β_sam[1, 1, N_Inf_burn:N_sam], [0.5, 0.025, 0.975]));
summary_table[2, :] = vcat(mean(β_sam[1, 2, N_Inf_burn:N_sam]),
    quantile(β_sam[1, 2, N_Inf_burn:N_sam], [0.5, 0.025, 0.975]));
summary_table[3, :] = vcat(mean(β_sam[2, 1, N_Inf_burn:N_sam]),
    quantile(β_sam[2, 1, N_Inf_burn:N_sam], [0.5, 0.025, 0.975]));
summary_table[4, :] = vcat(mean(β_sam[2, 2, N_Inf_burn:N_sam]),
    quantile(β_sam[2, 2, N_Inf_burn:N_sam], [0.5, 0.025, 0.975]));
summary_table[5, :] = vcat( mean(Σ_sam[1, 1, N_Inf_burn:N_sam]),
    quantile(Σ_sam[1, 1, N_Inf_burn:N_sam], [0.5, 0.025, 0.975]));
summary_table[6, :] = vcat(mean(Σ_sam[1, 2, N_Inf_burn:N_sam]),
    quantile(Σ_sam[1, 2, N_Inf_burn:N_sam], [0.5, 0.025, 0.975]));
summary_table[7, :] = vcat( mean(Σ_sam[2, 2, N_Inf_burn:N_sam]),
    quantile(Σ_sam[2, 2, N_Inf_burn:N_sam], [0.5, 0.025, 0.975]));
summary_table = round.(summary_table; digits = 6);
rnames = ["β[1, 1]", "β[1, 2]", "β[2, 1]", "β[2, 2]", "Σ[1, 1]", "Σ[1, 2]", "Σ[2, 2]"];
summary_table = [rnames summary_table];
pretty_table(summary_table,  ["" "mean" "median" "2.5%" "97.5%"], markdown)

|         |      mean |    median |      2.5% |     97.5% |
|---------|-----------|-----------|-----------|-----------|
| β[1, 1] |  0.251463 |  0.251457 |  0.251173 |  0.251762 |
| β[1, 2] |   0.13951 |  0.139511 |  0.139371 |  0.139655 |
| β[2, 1] | -0.133849 | -0.133846 | -0.134853 | -0.132716 |
| β[2, 2] |  0.060392 |  0.060399 |  0.059889 |  0.060899 |
| Σ[1, 1] |  0.015991 |   0.01599 |  0.015945 |  0.016039 |
| Σ[1, 2] | -0.006494 | -0.006494 | -0.006515 | -0.006474 |
| Σ[2, 2] |  0.003657 |  0.003657 |  0.003647 |  0.003668 |
